In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# One-hot encode the target variable
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

# Standardize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a simple neural network
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)

    def forward(self, X):
        self.hidden = np.dot(X, self.weights_input_hidden)
        self.hidden = self.sigmoid(self.hidden)
        output = np.dot(self.hidden, self.weights_hidden_output)
        return self.sigmoid(output)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def loss(self, predicted, actual):
        return np.mean((predicted - actual) ** 2)

    def backward(self, X, y, output, learning_rate=0.1):
        output_error = y - output
        output_delta = output_error * self.sigmoid_derivative(output)

        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden)

        # Update weights
        self.weights_hidden_output += self.hidden.T.dot(output_delta) * learning_rate
        self.weights_input_hidden += X.T.dot(hidden_delta) * learning_rate



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Define the PSO optimizer
class PSO:
    def __init__(self, nn, num_particles, max_iter, X, y):
        self.nn = nn
        self.num_particles = num_particles
        self.max_iter = max_iter
        self.X = X
        self.y = y

        # Initialize particles
        self.particles = [self.initialize_particle() for _ in range(num_particles)]
        self.global_best_position = self.particles[0]['position']
        self.global_best_loss = np.inf

    def initialize_particle(self):
        particle = {
            'position': {
                'input_hidden': np.random.randn(*self.nn.weights_input_hidden.shape),
                'hidden_output': np.random.randn(*self.nn.weights_hidden_output.shape),
            },
            'velocity': {
                'input_hidden': np.random.randn(*self.nn.weights_input_hidden.shape),
                'hidden_output': np.random.randn(*self.nn.weights_hidden_output.shape),
            },
            'best_position': None,
            'best_loss': np.inf,
        }
        return particle

    def update_velocity(self, particle):
        w = 0.5  # inertia weight
        c1 = 1.5  # cognitive coefficient
        c2 = 1.5  # social coefficient

        r1 = np.random.rand()
        r2 = np.random.rand()

        for layer in ['input_hidden', 'hidden_output']:
            cognitive_component = c1 * r1 * (particle['best_position'][layer] - particle['position'][layer])
            social_component = c2 * r2 * (self.global_best_position[layer] - particle['position'][layer])
            particle['velocity'][layer] = w * particle['velocity'][layer] + cognitive_component + social_component

    def update_position(self, particle):
        for layer in ['input_hidden', 'hidden_output']:
            particle['position'][layer] += particle['velocity'][layer]

    def optimize(self):
        for _ in range(self.max_iter):
            for particle in self.particles:
                # Set the weights of the neural network to the current particle's position
                self.nn.weights_input_hidden = particle['position']['input_hidden']
                self.nn.weights_hidden_output = particle['position']['hidden_output']

                # Calculate the loss
                predictions = self.nn.forward(self.X)
                loss = self.nn.loss(predictions, self.y)

                # Update the particle's best known position
                if loss < particle['best_loss']:
                    particle['best_loss'] = loss
                    particle['best_position'] = particle['position'].copy()

                # Update the global best position
                if loss < self.global_best_loss:
                    self.global_best_loss = loss
                    self.global_best_position = particle['position'].copy()

                # Update particle velocity and position
                self.update_velocity(particle)
                self.update_position(particle)

In [ ]:
# Function to save the weights after each epoch
def save_weights(epoch, nn):
    np.save(f'weights_input_hidden_epoch_{epoch}.npy', nn.weights_input_hidden)
    print("Input to hidden weights : ",nn.weights_input_hidden)
    np.save(f'weights_hidden_output_epoch_{epoch}.npy', nn.weights_hidden_output)
    print("Hidden to output weights : ",nn.weights_hidden_output)

# Example usage with epochs and saving weights
if __name__ == "__main__":
    # Initialize the neural network and PSO optimizer
    nn = NeuralNetwork(input_size=4, hidden_size=5, output_size=3)
    pso = PSO(nn, num_particles=20, max_iter=100, X=X_train, y=y_train)

    # Run the optimization with epochs and save weights
    epochs = 10
    for epoch in range(epochs):
        pso.optimize()

        # After each epoch, perform a backward pass to refine the weights
        for _ in range(10):  # Perform 10 steps of backpropagation after each epoch
            output = nn.forward(X_train)
            nn.backward(X_train, y_train, output)

        # Save weights of each neuron after each epoch
        save_weights(epoch, nn)
        print(f"Epoch {epoch + 1}/{epochs} complete")

    # Test the trained neural network on the test set
    predictions = nn.forward(X_test)
    test_loss = nn.loss(predictions, y_test)

    print("Test loss:", test_loss)

Input to hidden weights :  [[ 0.42171121  1.41930181 -2.43014888  0.3003442   2.19851654]
 [ 0.0956003   0.35025836  1.04737149 -5.77338272  1.45873103]
 [-4.45929078 -1.22717415 -0.41395803  0.29961942 -1.56566428]
 [-0.69496472 -1.12822077 -0.48472342  1.89368733  0.32301926]]
Hidden to output weights :  [[ 2.45121592e+00 -3.96313835e-01 -1.36808538e+00]
 [ 2.22676843e-01  2.11321703e-03 -1.63096209e+00]
 [ 9.44590690e-01 -4.57889272e-01 -4.07931527e+00]
 [-4.28574883e+00  8.32837441e-01  8.27229612e-01]
 [-1.71209569e-01 -1.51497856e+00  1.97478507e+00]]
Epoch 1/10 complete
Input to hidden weights :  [[ 0.60360438  1.27167906 -2.72341424  0.98240258  2.18608226]
 [ 0.3587174   0.18325364  0.46828137 -5.47318469  2.17115234]
 [-4.48592815 -1.68760981 -1.03219369  0.88583895 -1.05294691]
 [-0.73626722 -1.70597544 -1.51133431  2.50374818  1.50835292]]
Hidden to output weights :  [[ 2.81758817 -0.50803944 -2.43997986]
 [-0.07412938  0.16612391 -2.91564952]
 [ 0.97265889 -0.62349286 -4.7

In [ ]:
# Same with accuracy
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# One-hot encode the target variable
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

# Standardize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a simple neural network
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)

    def forward(self, X):
        self.hidden = np.dot(X, self.weights_input_hidden)
        self.hidden = self.sigmoid(self.hidden)
        output = np.dot(self.hidden, self.weights_hidden_output)
        return self.sigmoid(output)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def loss(self, predicted, actual):
        return np.mean((predicted - actual) ** 2)

    def backward(self, X, y, output, learning_rate=0.1):
        output_error = y - output
        output_delta = output_error * self.sigmoid_derivative(output)

        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden)

        # Update weights
        self.weights_hidden_output += self.hidden.T.dot(output_delta) * learning_rate
        self.weights_input_hidden += X.T.dot(hidden_delta) * learning_rate

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Define the PSO optimizer
class PSO:
    def __init__(self, nn, num_particles, max_iter, X, y):
        self.nn = nn
        self.num_particles = num_particles
        self.max_iter = max_iter
        self.X = X
        self.y = y

        # Initialize particles
        self.particles = [self.initialize_particle() for _ in range(num_particles)]
        self.global_best_position = self.particles[0]['position']
        self.global_best_loss = np.inf

    def initialize_particle(self):
        particle = {
            'position': {
                'input_hidden': np.random.randn(*self.nn.weights_input_hidden.shape),
                'hidden_output': np.random.randn(*self.nn.weights_hidden_output.shape),
            },
            'velocity': {
                'input_hidden': np.random.randn(*self.nn.weights_input_hidden.shape),
                'hidden_output': np.random.randn(*self.nn.weights_hidden_output.shape),
            },
            'best_position': None,
            'best_loss': np.inf,
        }
        return particle

    def update_velocity(self, particle):
        w = 0.5  # inertia weight
        c1 = 1.5  # cognitive coefficient
        c2 = 1.5  # social coefficient

        r1 = np.random.rand()
        r2 = np.random.rand()

        for layer in ['input_hidden', 'hidden_output']:
            cognitive_component = c1 * r1 * (particle['best_position'][layer] - particle['position'][layer])
            social_component = c2 * r2 * (self.global_best_position[layer] - particle['position'][layer])
            particle['velocity'][layer] = w * particle['velocity'][layer] + cognitive_component + social_component

    def update_position(self, particle):
        for layer in ['input_hidden', 'hidden_output']:
            particle['position'][layer] += particle['velocity'][layer]

    def optimize(self):
        for _ in range(self.max_iter):
            for particle in self.particles:
                # Set the weights of the neural network to the current particle's position
                self.nn.weights_input_hidden = particle['position']['input_hidden']
                self.nn.weights_hidden_output = particle['position']['hidden_output']

                # Calculate the loss
                predictions = self.nn.forward(self.X)
                loss = self.nn.loss(predictions, self.y)

                # Update the particle's best known position
                if loss < particle['best_loss']:
                    particle['best_loss'] = loss
                    particle['best_position'] = particle['position'].copy()

                # Update the global best position
                if loss < self.global_best_loss:
                    self.global_best_loss = loss
                    self.global_best_position = particle['position'].copy()

                # Update particle velocity and position
                self.update_velocity(particle)
                self.update_position(particle)

In [ ]:
# Training and Evaluation
if __name__ == "__main__":
    # Initialize the neural network and PSO optimizer
    nn = NeuralNetwork(input_size=4, hidden_size=5, output_size=3)
    pso = PSO(nn, num_particles=20, max_iter=100, X=X_train, y=y_train)

    # Run the optimization with epochs
    epochs = 10
    for epoch in range(epochs):
        pso.optimize()

        # After each epoch, perform a backward pass to refine the weights
        for _ in range(10):  # Perform 10 steps of backpropagation after each epoch
            output = nn.forward(X_train)
            nn.backward(X_train, y_train, output)

        print(f"Epoch {epoch + 1}/{epochs} complete")

    # Save the final weights after training
    best_weights_input_hidden = nn.weights_input_hidden
    best_weights_hidden_output = nn.weights_hidden_output
    print("Final weights of each neuron:")
    print("Input to Hidden Layer Weights:\n", best_weights_input_hidden)
    print("Hidden to Output Layer Weights:\n", best_weights_hidden_output)

    # Test the trained neural network on the test set
    predictions = nn.forward(X_test)
    predictions = np.argmax(predictions, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(y_test_labels, predictions)
    print("Test Accuracy:", accuracy)

Epoch 1/10 complete
Epoch 2/10 complete
Epoch 3/10 complete
Epoch 4/10 complete
Epoch 5/10 complete
Epoch 6/10 complete
Epoch 7/10 complete
Epoch 8/10 complete
Epoch 9/10 complete
Epoch 10/10 complete
Final weights of each neuron:
Input to Hidden Layer Weights:
 [[-0.38841055  0.71173248 -1.23697273 -1.38642165 -0.24523533]
 [ 0.40131432 -3.0698096   4.25463793  0.67368792  0.58492827]
 [-0.87503074  5.36092339 -4.41104296  1.80951761 -0.23097126]
 [-0.93342123  0.82991196 -2.61323533  2.68240349 -1.66470785]]
Hidden to Output Layer Weights:
 [[ 1.30107293  2.97373395 -4.44862273]
 [-4.47961686  3.47453084 -0.54869815]
 [ 3.93056968 -6.45192463 -3.2802569 ]
 [-3.62434213 -5.43239609  3.56255753]
 [-0.53067419  1.54518979 -6.39713624]]
Test Accuracy: 0.9666666666666667


In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# One-hot encode the target variable
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

# Standardize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a simple neural network
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)

    def forward(self, X):
        self.hidden = np.dot(X, self.weights_input_hidden)
        self.hidden = self.sigmoid(self.hidden)
        output = np.dot(self.hidden, self.weights_hidden_output)
        return self.sigmoid(output)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def loss(self, predicted, actual):
        return np.mean((predicted - actual) ** 2)

    def accuracy(self, predicted, actual):
        predicted_labels = np.argmax(predicted, axis=1)
        actual_labels = np.argmax(actual, axis=1)
        return np.mean(predicted_labels == actual_labels)

    def backward(self, X, y, output, learning_rate=0.1):
        output_error = y - output
        output_delta = output_error * self.sigmoid_derivative(output)

        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden)

        # Update weights
        self.weights_hidden_output += self.hidden.T.dot(output_delta) * learning_rate
        self.weights_input_hidden += X.T.dot(hidden_delta) * learning_rate

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Define the PSO optimizer
class PSO:
    def __init__(self, nn, num_particles, max_iter, X, y):
        self.nn = nn
        self.num_particles = num_particles
        self.max_iter = max_iter
        self.X = X
        self.y = y

        # Initialize particles
        self.particles = [self.initialize_particle() for _ in range(num_particles)]
        self.global_best_position = self.particles[0]['position']
        self.global_best_loss = np.inf

    def initialize_particle(self):
        particle = {
            'position': {
                'input_hidden': np.random.randn(*self.nn.weights_input_hidden.shape),
                'hidden_output': np.random.randn(*self.nn.weights_hidden_output.shape),
            },
            'velocity': {
                'input_hidden': np.random.randn(*self.nn.weights_input_hidden.shape),
                'hidden_output': np.random.randn(*self.nn.weights_hidden_output.shape),
            },
            'best_position': None,
            'best_loss': np.inf,
        }
        return particle

    def update_velocity(self, particle):
        w = 0.5  # inertia weight
        c1 = 1.5  # cognitive coefficient
        c2 = 1.5  # social coefficient

        r1 = np.random.rand()
        r2 = np.random.rand()

        for layer in ['input_hidden', 'hidden_output']:
            cognitive_component = c1 * r1 * (particle['best_position'][layer] - particle['position'][layer])
            social_component = c2 * r2 * (self.global_best_position[layer] - particle['position'][layer])
            particle['velocity'][layer] = w * particle['velocity'][layer] + cognitive_component + social_component

    def update_position(self, particle):
        for layer in ['input_hidden', 'hidden_output']:
            particle['position'][layer] += particle['velocity'][layer]

    def optimize(self):
        for _ in range(self.max_iter):
            for particle in self.particles:
                # Set the weights of the neural network to the current particle's position
                self.nn.weights_input_hidden = particle['position']['input_hidden']
                self.nn.weights_hidden_output = particle['position']['hidden_output']

                # Calculate the loss
                predictions = self.nn.forward(self.X)
                loss = self.nn.loss(predictions, self.y)

                # Update the particle's best known position
                if loss < particle['best_loss']:
                    particle['best_loss'] = loss
                    particle['best_position'] = particle['position'].copy()

                # Update the global best position
                if loss < self.global_best_loss:
                    self.global_best_loss = loss
                    self.global_best_position = particle['position'].copy()

                # Update particle velocity and position
                self.update_velocity(particle)
                self.update_position(particle)

In [ ]:
# Example usage with epochs
if __name__ == "__main__":
    # Initialize the neural network and PSO optimizer
    nn = NeuralNetwork(input_size=4, hidden_size=5, output_size=3)
    pso = PSO(nn, num_particles=20, max_iter=100, X=X_train, y=y_train)

    # Run the optimization with epochs
    epochs = 10
    for epoch in range(epochs):
        pso.optimize()

        # After each epoch, perform a backward pass to refine the weights
        for _ in range(10):  # Perform 10 steps of backpropagation after each epoch
            output = nn.forward(X_train)
            nn.backward(X_train, y_train, output)

        # Save weights after each epoch
        np.save(f'weights_input_hidden_epoch_{epoch}.npy', nn.weights_input_hidden)
        print("Input to hidden : ",nn.weights_input_hidden)
        np.save(f'weights_hidden_output_epoch_{epoch}.npy', nn.weights_hidden_output)
        print("Hidden to output : ", nn.weights_hidden_output)

        # Calculate accuracy
        predictions = nn.forward(X_test)
        accuracy = nn.accuracy(predictions, y_test)
        print(f"Epoch {epoch + 1}/{epochs}, Accuracy: {accuracy * 100:.2f}%")

    # Final test loss and accuracy
    predictions = nn.forward(X_test)
    test_loss = nn.loss(predictions, y_test)
    test_accuracy = nn.accuracy(predictions, y_test)

    print("Final Test loss:", test_loss)
    print("Final Test accuracy:", test_accuracy * 100, "%")

Input to hidden :  [[-0.25264669 -0.69681988  0.33272721  0.38724521 -2.62785346]
 [-1.17553141  0.25937668  0.2222974  -1.01278395 -0.46192258]
 [ 2.23943106  1.7203973   2.05641372 -0.37553394 -2.37585509]
 [ 1.95065035  1.68930211 -0.74522366 -2.13866962 -0.99714598]]
Hidden to output :  [[-3.01351443  0.08301822 -1.88256106]
 [-0.68960396  0.38167036  1.57796096]
 [-0.31145617 -0.85431881  2.12294858]
 [-0.29163233  0.06608375 -2.14107325]
 [ 2.03880414 -0.71511894 -1.5317172 ]]
Epoch 1/10, Accuracy: 90.00%
Input to hidden :  [[-0.02810958 -1.00366896  0.06825572  0.75286987 -2.29027026]
 [-1.92976141 -0.15408346  0.76090475 -0.64214536  0.76348708]
 [ 2.63087367  1.72842998  2.21239534 -0.51946416 -2.56631925]
 [ 2.1773357   1.77070088  0.11278722 -2.40874962 -1.08443121]]
Hidden to output :  [[-4.0432121   1.09963444 -1.78421864]
 [-1.29715098 -0.21665708  1.97919737]
 [-0.86719036 -1.43220249  1.84628562]
 [ 0.03605846  0.53936041 -3.36604586]
 [ 2.50970877 -1.57353163 -2.513058

In [ ]:
!pip install tensorflow-model-optimization --upgrade

In [ ]:
!pip install pyswarm

  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4464 sha256=8d995a6db30435de908baa409682fd9a0b651df3fb3a4b25cce87b78b97cc69f
  Stored in directory: /root/.cache/pip/wheels/71/67/40/62fa158f497f942277cbab8199b05cb61c571ab324e67ad0d6
Successfully built pyswarm


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pyswarm import pso
import h5py

# Load Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# One-hot encode the target labels
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# PSO Objective Function
def pso_objective(params,weights_filepath):
    lr = params[0]
    units1 = int(params[1])
    units2 = int(params[2])

    # Build neural network model
    model = Sequential()
    model.add(Dense(units1, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(units2, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # Output layer (3 classes)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    class WeightsSaver(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            with h5py.File(weights_filepath, 'a') as f:
                group = f.create_group(f'epoch_{epoch + 1}')
                for layer_idx, layer in enumerate(self.model.layers):
                    weights = layer.get_weights()
                    group.create_dataset(f'layer_{layer_idx}_weights', data=weights[0])
                    group.create_dataset(f'layer_{layer_idx}_biases', data=weights[1])

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0, callbacks=[WeightsSaver()])

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

    return -accuracy  # PSO minimizes the objective function, so return negative accuracy

In [ ]:
# PSO parameters and bounds
lb = [1e-5, 16, 16]  # Lower bounds: learning rate, units1, units2
ub = [1e-2, 128, 128]  # Upper bounds

# Save the weights for each epoch to a .h5 file
weights_filepath = 'model_weights.h5'

# Run PSO to find optimal hyperparameters
best_params, best_accuracy = pso(pso_objective(lb, weights_filepath), lb, ub, swarmsize=10, maxiter=5)

# Print the best parameters and accuracy
print(f"Best Parameters: Learning rate={best_params[0]}, Units1={best_params[1]}, Units2={best_params[2]}")
print(f"Best Accuracy achieved: {-best_accuracy}")

# Load and print weights from the HDF5 file (optional, just for checking)
with h5py.File('model_weights.h5', 'r') as f:
    for epoch in range(1, 11):
        print(f"\nWeights from epoch {epoch}:")
        epoch_group = f[f'epoch_{epoch}']
        for layer in epoch_group:
            print(f"{layer}: {epoch_group[layer][()]}")  # Print the weights for each layer

ValueError: Unable to synchronously create group (name already exists)

In [ ]:
# 1 Line Description for the below code
## The code builds and trains a 3-layer ANN on the IMDb dataset for binary sentiment classification, saving model weights after each epoch.
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

# Load and preprocess the IMDb dataset
max_features = 10000  # Vocabulary size
maxlen = 200  # Max length of the review

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

# Create the directory to save weights if it doesn't exist
if not os.path.exists('saved_weights'):
    os.makedirs('saved_weights')

# Define the ANN model with 3 layers
def create_ann():
    model = Sequential()
    model.add(Embedding(max_features, 128, input_length=maxlen))
    model.add(Flatten())  # Converts 3D sequence to 2D
    model.add(Dense(64, activation='relu'))  # First hidden layer
    model.add(Dense(1, activation='sigmoid'))  # Output layer (binary classification)

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Custom callback to save weights after each epoch
class SaveWeightsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Save model weights at the end of each epoch
        weight_file = f"saved_weights/weights_epoch_{epoch+1}.weights.h5"
        self.model.save_weights(weight_file)
        print(f"Saved weights for epoch {epoch+1} to {weight_file}")

# Create and compile the model
model = create_ann()

# Train the model for 10 epochs and save weights after each epoch
save_weights_callback = SaveWeightsCallback()
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test), callbacks=[save_weights_callback])

# Evaluate final model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Final accuracy: {accuracy}, Final loss: {loss}")


Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7076 - loss: 0.5125Saved weights for epoch 1 to saved_weights/weights_epoch_1.weights.h5
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7079 - loss: 0.5122 - val_accuracy: 0.8599 - val_loss: 0.3231
Epoch 2/10
376/391 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9719 - loss: 0.0871Saved weights for epoch 2 to saved_weights/weights_epoch_2.weights.h5
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9719 - loss: 0.0869 - val_accuracy: 0.8479 - val_loss: 0.4170
Epoch 3/10
373/391 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9979 - loss: 0.0113Saved weights for epoch 3 to saved_weights/weights_epoch_3.weights.h5
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9979 - loss: 0.0112 - val_accuracy: 0.8460 - val_loss: 0.5243
Epoch 4/10
387/391 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9998 - loss: 0.0016Saved weights for epoch 4 to saved_weights/weights_epoch_4.weights.h5
391/391 ━━━━━━━━━━━━━━━━━━━━ 